In [1]:
from io import StringIO
import sys, getopt
import pandas as pd
import numpy as np
import re
import os

In [2]:
import glob
text = ""
for i in glob.glob("Data/" + "*.txt"):
    with open(i, 'r') as f:
        text += f.read()

In [3]:
len(text)

5667777

In [4]:
alfabe = 'çığöşüâqwertyuioplkjhgfdsazxcvbnmÇIĞÖŞÜÂQWERTYUIOPLKJHGFDSAZXCVBNM'
alfabe = list(alfabe)
alfabe.append("<start>")
alfabe.append("<end>")
alfabe.append("\n")
alfabe.append("<")
alfabe.append(">")
alfabe.append("\\")
alfabe.append(" ")

In [5]:
chars_set = set(text)
for i in chars_set:
    if i not in alfabe:
        text = text.replace(i, "")

In [6]:
text = text.replace("\n", " \n ")
text = text.replace("<start>", " <start> ")
text = text.replace("<end>", " <end> ")

In [7]:
for k in range(100):
    text = text.replace("  ", " ")
    text = text.replace("   ", " ")
    text = text.replace("    ", " ")

In [8]:
text = text.replace("<start>", "<siir> <start>")
text = text.replace("<end>", "<end> <zero> <zero> <zero> <zero>")

In [9]:
text = text.split("<siir>")

In [10]:
for i in range(len(text)):
    text[i] = text[i].split(" ")

In [12]:
import gensim.models.word2vec as w2v
import multiprocessing
num_features = 300
min_word_count = 1
num_workers = multiprocessing.cpu_count()
context_size = 7
downsampling = 1e-3
seed = 1

w2vmodel = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

In [13]:
w2vmodel.build_vocab(text)

In [14]:
w2vmodel.train(text, total_examples=w2vmodel.corpus_count, epochs=20)

(15352169, 20134540)

In [15]:
w2vmodel.wv.most_similar('<start>', topn=10)

[('', 0.7671248912811279),
 ('I', 0.43655478954315186),
 ('mevsiminde', 0.3820810616016388),
 ('tutunuşu', 0.34387272596359253),
 ('Yeşilçiçeği', 0.34146276116371155),
 ('bulutlarıyla', 0.33917298913002014),
 ('sarsılmadan', 0.33483457565307617),
 ('Günlerden', 0.3341832160949707),
 ('Saide', 0.3322668969631195),
 ('Kanağacı', 0.33092260360717773)]

In [16]:
word_to_vec_map = w2vmodel[w2vmodel.wv.vocab]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [17]:
vocab = w2vmodel.wv.vocab

In [18]:
word_to_index={}
index_to_word = {}
for i, w in enumerate(vocab):
    word_to_index[w] = i
    index_to_word[i] = w

In [19]:
word_to_index['bu']

812

In [20]:
import tflearn
from tflearn.data_utils import *
import tensorflow as tf 

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [21]:
max_len = 5

In [22]:
g = tflearn.input_data([None, max_len, num_features])
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512)
g = tflearn.dropout(g, 0.5)
g = tflearn.fully_connected(g, num_features, activation='linear')
g = tflearn.regression(g, optimizer='adam', loss='mean_square',
                       learning_rate=0.001)

In [23]:
from tqdm import tqdm

trainX = []
trainY = []
for i in tqdm(text):
    if len(i) > 6:
        for k in range(len(i)-6):
            
            a = []
            for j in range(k,k+5):
                a.append(w2vmodel[i[j]])
            trainX.append(a)
            
            b= w2vmodel[i[k+5]]
            
            trainY.append(b)

  0%|          | 0/7387 [00:00<?, ?it/s]/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
100%|██████████| 7387/7387 [00:31<00:00, 231.80it/s]


In [24]:
len(trainX)

962409

In [25]:
trainX = np.array(trainX)

In [26]:
trainY = np.array(trainY)

In [27]:
trainX.shape, trainY.shape

((962409, 5, 300), (962409, 300))

In [ ]:
model = tflearn.DNN(g, tensorboard_verbose=0)

In [ ]:
for step in range(100):
  model.fit(trainX, trainY, validation_set=0.1, n_epoch=1, run_id='siir')
  if step % 5 == 0:
    model.save(str(step) + '2.tflearn')

Training Step: 13533  | total loss: 0.07436 | time: 12688.225s
| Adam | epoch: 001 | loss: 0.07436 -- iter: 866112/866168


In [0]:
prediction = model.predict(trainX)[0]

In [0]:
w2vmodel.wv.similar_by_vector(prediction)

In [0]:
w2vmodel.wv.similar_by_vector(trainX[0][0],topn=1)[0][0], w2vmodel.wv.similar_by_vector(trainX[0][1],topn=1)

In [0]:
for i in range(5):
    print(w2vmodel.wv.similar_by_vector(trainX[0][i],topn=1)[0][0])